In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd ='C:/Program Files/Tesseract-OCR/tesseract.exe'
from googletrans import Translator
from langdetect import detect
import re

In [36]:
input_string= 'D:/Thesis-20230616T142242Z-001/Thesis/Pic/10.jpg'
dataset_csv = 'D:/Thesis-20230616T142242Z-001/Thesis/medicine..csv'
dataset_image = 'D:/Thesis-20230616T142242Z-001/Thesis/image_dataset'

In [37]:
image = Image.open(input_string)

In [34]:
textOfImage = pytesseract.image_to_string(image, lang='eng') # for all latin languages
# textOfImage = pytesseract.image_to_string(image, lang='chi_sim') #for non-latin chinease languages 
clean_text = remove_special_characters(textOfImage)
print(get_first_word(clean_text))
text_after_word= get_first_word(clean_text)
# english_string = convert_to_english(textOfImage)
print(text_after_word)


NameError: name 'remove_special_characters' is not defined

In [29]:
print(textOfImage)

Sedno”

‘Dettocatadine



In [52]:
flagForText = 0
target_word = 'Name: '
if target_word in textOfImage:
    # Find the index of the target word
    index = textOfImage.index(target_word)
    flagForText = 1;
    # Print everything after the target word
    text_after_word = textOfImage[index + len(target_word):]
    print("Text after the word:", text_after_word)
else:
    print("Word not found.")

Text after the word: A-Cal DX



In [55]:
print(text_after_word)

A-Cal DX



In [56]:
if flagForText == 1:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier
    import scipy.sparse
    from sklearn.multioutput import ClassifierChain
    from sklearn.naive_bayes import MultinomialNB
    import pyttsx3
    
    dataset_path = dataset_csv
    data = pd.read_csv(dataset_path)
    data = data.dropna()
    
    X= data['brand name']  # Input feature: Medicine Name
    y = data[['type', 'slug', 'dosage form', 'generic', 'strength']]  # Target variables: Purpose columns
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    vectorizer = TfidfVectorizer()
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)
    
    # Convert the vectorized data to a sparse matrix
    X_train_vectorized = scipy.sparse.csr_matrix(X_train_vectorized)
    X_test_vectorized = scipy.sparse.csr_matrix(X_test_vectorized)
    
    # Train the Random Forest model with optimized memory usage
    model = RandomForestClassifier(n_estimators=100, max_depth=10)
    model.fit(X_train_vectorized, y_train)
    
    medicine_name = text_after_word
    medicine_name_vectorized = vectorizer.transform([medicine_name])
    medicine_name_vectorized = scipy.sparse.csr_matrix(medicine_name_vectorized)
    predicted_purposes = model.predict(medicine_name_vectorized)
    
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    
    # Output the predictions
    for column, purpose in zip(y.columns, predicted_purposes[0]):
            text =f'the {column} is {purpose}' 
            # Use the engine to speak the text
            engine.say(text)
            engine.runAndWait()
            print(f'Predicted Purpose {column}: {purpose}')
else:
    import tensorflow as tf
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
    from tensorflow.keras.optimizers import Adam
    import pyttsx3
    
    # Set up the data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    
    # Load and preprocess the training images
    train_generator = datagen.flow_from_directory(
        dataset_image,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical'
    )
    
    # Define the model architecture
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(train_generator.num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    #if model is already created then uncomment this block
    #model = tf.keras.models.load_model('image_classification_model.h5') 
    
    # Train the model
    model.fit(train_generator, epochs=10)

    # Save the model ((( to create new model uncomment this line)))
    model.save('image_classification_model.h5')
    
    # Predict the details of new images
    image_path = input_string
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0
    image = tf.expand_dims(image, 0)
    
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    
    predictions = model.predict(image)
    class_index = tf.argmax(predictions[0])
    class_label = train_generator.class_indices
    for key, value in class_label.items():
        if value == class_index:
            text =f'Image details is, {key}'; 
            # Use the engine to speak the text
            engine.say(text)
            engine.runAndWait()
            print("Image details:", key)
            break

Predicted Purpose type: allopathic
Predicted Purpose slug: a-cal-dxtablet500-mg400-iu
Predicted Purpose dosage form: Tablet
Predicted Purpose generic: Calcium Carbonate [Elemental source] + Vitamin D3
Predicted Purpose strength: 600 mg+400 IU
